In [2]:
from gcl_sem.libs.mapillary_sls.mapillary_sls.datasets.msls import MSLS
msls_root="/deepstore/datasets/dmb/ComputerVision/nis-data/acvpr-msls"
msls_dataset = MSLS(msls_root, "", mode="val", posDistThr=25)



=====> cph
=====> sf


/home/s3155900/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [3]:
def key_from_path(path):
    return path.split("/")[-1].split(".")[0]

In [4]:
query_keys = [
    key_from_path(msls_dataset.qImages[q_idx]) for q_idx in msls_dataset.qIdx
]

positive_keys = [
    [key_from_path(msls_dataset.dbImages[pos_idx]) for pos_idx in pos_indexes]
    for pos_indexes in msls_dataset.pIdx
]

print(len(query_keys))

740


In [5]:
# top_k_file = "/home/s3155900/gregory/utwente-acvpr-gcl_sem/runs/gcl_channels/run_3/top_k_20.json"
    # "/home/s3155900/gregory/utwente-acvpr-gcl_sem/runs/gcl_channels/run_3/top_k_10.json"
top_k_file = "/home/s3155900/gregory/utwente-acvpr-gcl_sem/runs/gcl_normal/run_2/top_k_0.txt"


with open(top_k_file) as f:
    preds = {
        query: matches
        for query, *matches in [line.strip().split() for line in f.readlines()]
    }


import os
import numpy as np

cph_path = "/deepstore/datasets/dmb/ComputerVision/nis-data/acvpr-msls/train_val/cph"
sf_path = "/deepstore/datasets/dmb/ComputerVision/nis-data/acvpr-msls/train_val/sf"

# key to image path
cph_images = {}
for root, dirs, files in os.walk(cph_path):
    for file in files:
        if file.endswith(".jpg"):
            # get filename without path and extension
            key = os.path.splitext(os.path.basename(file))[0]

            cph_images[key] = os.path.join(root, file)

sf_images = {}
for root, dirs, files in os.walk(sf_path):
    for file in files:
        if file.endswith(".jpg"):
            # get filename without path and extension
            key = os.path.splitext(os.path.basename(file))[0]

            sf_images[key] = os.path.join(root, file)


all_images = {**cph_images, **sf_images}

accurate_matches = {
    query: [match for match in matches if match in positive_keys[i]]
    for i, (query, matches) in enumerate(preds.items())
}

query_keys_sorted_by_nb_matches = sorted(
    accurate_matches.keys(), key=lambda k: len(accurate_matches[k]), reverse=True
)


recall = []
# for each predictions, check if any of the first 20 matches is in the positive keys
for query, matches in preds.items():
    query_idx = query_keys.index(query)

    has_match = False
    for match in matches[0:20]:
        if match in positive_keys[query_idx]:
            has_match = True
            break


    if has_match:
        recall.append(1)
    else:
        recall.append(0)

print("Recall: ", np.mean(recall))

    

Recall:  0.7094594594594594


In [6]:
from PIL import Image
import matplotlib.pyplot as plt
from code import interact

def show_top_k(index=0):
    query = query_keys_sorted_by_nb_matches[index]
    matches = preds[query]

    query_image = all_images[query]
    
    correct_matches = accurate_matches[query]

    print(f"Query image: {query}")
    print(f"Correct matches: {len(correct_matches)}")
    for c in correct_matches:
        print(f"  {c}")
    
    img = Image.open(query_image)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

    # create a subplot for each match
    fig, axs = plt.subplots(5, 5)

    # make figure bigger
    fig.set_figwidth(20)
    fig.set_figheight(15)

    # find the matches
    for i, match in enumerate(matches):
        if match in all_images:
            match_image = all_images[match]
        else:
            print(f"Could not find image for {match}")
            continue
        
        img = Image.open(match_image)
        img = np.array(img)

        border_size = 20
        if match in correct_matches:
            axs[i // 5, i % 5].set_title(f"Correct")

            border_color = [0, 255, 0]
        else:
            axs[i // 5, i % 5].set_title("Incorrect")
            border_color = [255, 0, 0]

        img[:border_size, :, :] = border_color
        img[-border_size:, :, :] = border_color
        img[:, :border_size, :] = border_color
        img[:, -border_size:, :] = border_color
        
        axs[i // 5, i % 5].imshow(img)

        # remove axis
        axs[i // 5, i % 5].axis("off")

    plt.show()

from ipywidgets import interact

interact(show_top_k, index=(0, len(preds) - 1))

interactive(children=(IntSlider(value=0, description='index', max=739), Output()), _dom_classes=('widget-inter…

<function __main__.show_top_k(index=0)>